In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mlt
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
df

In [ ]:
df['Class'].value_counts()

In [ ]:
df.hist(bins= 30 , figsize = (30,30))

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import RobustScaler
new_df = df.copy()
new_df['Amount'] = RobustScaler().fit_transform(new_df['Amount'].to_numpy().reshape(-1,1))
new_df['Amount'].hist()
time = new_df['Time']
new_df['Time'] = (time - time.min())/(time.max() - time.min())
new_df


In [ ]:
new_df = new_df.sample(frac = 1 , random_state = 1)
new_df


In [ ]:
train , test, val = new_df[:240000], new_df[240000:262000], new_df[262000:]
train['Class'].value_counts(), test['Class'].value_counts(), val['Class'].value_counts()

In [ ]:
train_np, test_np, val_np = train.to_numpy(), test.to_numpy(),val.to_numpy()
train_np.shape, test_np.shape, val_np.shape

In [ ]:
X_train, y_train = train_np[:,:-1] , train_np[:,-1]
X_test, y_test = test_np[:,:-1] , test_np[:,-1]
X_val, y_val = val_np[:,:-1] , val_np[:,-1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_model.score(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, logistic_model.predict(X_val), target_names= ['Not Fraud', 'Fraud']))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint


shallow_nn = Sequential()
shallow_nn.add(InputLayer((X_train.shape[1],)))
shallow_nn.add(Dense(2,'relu'))
shallow_nn.add(BatchNormalization())
shallow_nn.add(Dense(1, 'sigmoid'))


checkpoint = ModelCheckpoint('shallow_nn.keras', save_best_only = True)
shallow_nn.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])



In [ ]:
shallow_nn.summary()

In [ ]:
shallow_nn.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 5, callbacks= checkpoint)

In [ ]:
def neural_net_predictions(model, x):
    return (shallow_nn.predict(x).flatten() > 0.5).astype(int)
neural_net_predictions(shallow_nn, X_val)


In [ ]:
print(classification_report(y_val, neural_net_predictions(shallow_nn, X_val), target_names = ['Not Frud', 'Fraud']))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 2 , n_jobs = -1)
rf.fit(X_train, y_train)
print(classification_report(y_val, rf.predict(X_val), target_names = ['Not Frud', 'Fraud']))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier (n_estimators = 50, learning_rate = 1.0, max_depth = 1, random_state = 0)
gbc.fit(X_train, y_train)
print(classification_report(y_val, gbc.predict(X_val), target_names = ['Not Fraud' , 'Fraud']))

In [ ]:
from sklearn.svm import LinearSVC
svc = LinearSVC(class_weight = 'balanced')
svc.fit(X_train, y_train)
print(classification_report(y_val, svc.predict(X_val), target_names = ['Not Fraud' , 'Fraud']))


In [ ]:
new_df.head()

In [ ]:
not_frauds = new_df.query('Class == 0')
frauds = new_df.query('Class == 1')
not_frauds['Class'].value_counts(), frauds['Class'].value_counts()



In [ ]:
balanced_df = pd.concat([frauds, not_frauds.sample(len(frauds), random_state = 1)])
balanced_df['Class'].value_counts()


In [ ]:
balanced_df = balanced_df.sample(frac = 1, random_state = 1)
balanced_df

In [ ]:
balanced_df_np = balanced_df.to_numpy()
X_train_b, y_train_b = balanced_df_np[:700, :-1],balanced_df_np[:700, -1]
X_test_b, y_test_b = balanced_df_np[700:842, :-1],balanced_df_np[700:842, -1]
X_val_b, y_val_b = balanced_df_np[842:, :-1],balanced_df_np[842:, -1]
X_train_b.shape, y_train_b.shape, X_test_b.shape, y_test_b.shape, X_val_b.shape, y_val_b.shape

In [ ]:
pd.Series(y_train_b).value_counts(), pd.Series(y_test_b).value_counts(),pd.Series(y_val_b).value_counts()

In [ ]:
logistic_model_b = LogisticRegression()
logistic_model_b.fit(X_train_b, y_train_b)
logistic_model_b.score(X_train_b, y_train_b)
print(classification_report(y_val_b, logistic_model_b.predict(X_val_b), target_names = ['Not Frud', 'Fraud']))

In [ ]:
shallow_nn_b = Sequential()
shallow_nn_b.add(InputLayer((X_train.shape[1],)))
shallow_nn_b.add(Dense(2,'relu'))
shallow_nn_b.add(BatchNormalization())
shallow_nn_b.add(Dense(1, 'sigmoid'))


checkpoint = ModelCheckpoint('shallow_nn.keras', save_best_only = True)
shallow_nn_b.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
shallow_nn_b.fit(X_train_b, y_train_b, validation_data = (X_val_b, y_val_b), epochs = 40, callbacks= checkpoint)

In [ ]:
shallow_nn_b.fit(X_train_b, y_train_b, validation_data = (X_val_b, y_val_b), epochs = 40, callbacks= checkpoint)

In [ ]:
print(classification_report(y_val_b, neural_net_predictions(shallow_nn_b, X_val_b), target_names = ['Not Frud', 'Fraud']))

In [ ]:
shallow_nn_b1 = Sequential()
shallow_nn_b1.add(InputLayer((X_train.shape[1],)))
shallow_nn_b1.add(Dense(1,'relu'))
shallow_nn_b1.add(BatchNormalization())
shallow_nn_b1.add(Dense(1, 'sigmoid'))


checkpoint = ModelCheckpoint('shallow_nn.keras', save_best_only = True)
shallow_nn_b1.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
shallow_nn_b1.fit(X_train_b, y_train_b, validation_data = (X_val_b, y_val_b), epochs = 40, callbacks= checkpoint)

In [ ]:
print(classification_report(y_val_b, neural_net_predictions(shallow_nn_b, X_val_b), target_names = ['Not Frud', 'Fraud']))

In [ ]:
gbc_b = GradientBoostingClassifier (n_estimators = 50, learning_rate = 1.0, max_depth = 1, random_state = 0)
gbc_b.fit(X_train_b, y_train_b)
print(classification_report(y_val_b, gbc.predict(X_val_b), target_names = ['Not Fraud' , 'Fraud']))

In [ ]:
svc_b = LinearSVC(class_weight = 'balanced')
svc_b.fit(X_train, y_train)
print(classification_report(y_val_b, svc.predict(X_val_b), target_names = ['Not Fraud' , 'Fraud']))


In [ ]:
print(classification_report(y_test_b, neural_net_predictions(shallow_nn_b1, X_test_b), target_names = ['Not Frud', 'Fraud']))